In [25]:
import expipe
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tools
from tqdm import tqdm
import copy
import seaborn as sns
from scipy import stats
import datetime as dt
from matplotlib import colors
import datetime as dt
import warnings

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Params

In [22]:
p_phase = {
   'Criteria_LatentPhase': {
       'min_len': dt.timedelta(days=3),  # minimal length
       'max_time_latent': dt.timedelta(days=55) # maximal length
   },
   'BL_0': {
       'len': dt.timedelta(days=3),
       'color': '#FDE725FF'},
   'BL_1': {
       'len': dt.timedelta(days=3),
       'color': '#C7E020FF'},
   'Latent_0': {
       'len': dt.timedelta(days=3),
       'color': '#8FD744FF'},
   'Latent_1': {
       'len': dt.timedelta(days=3),
       'color': '#75D054FF'},   
   'Latent_2': {
       'len': dt.timedelta(days=3),
       'color': '#47C16EFF'},
   'Chronic_0': {
       'len': dt.timedelta(days=3),
       'color': '#27AD81FF'},
   'Chronic_1': {
       'len': dt.timedelta(days=3),
       'color': '#1F9A8AFF'},
   'Chronic_2': {
       'len': dt.timedelta(days=3),
       'color': '#24868EFF'},
}

p_columnnames = {
    'date': 'Date',
    'pps_stim': 'PPS Stimulation',
    'surgery': 'Surgery'
}
p_dat = {
    'path_expipe': "/home/jovyan/work/data_expipe/epimirna/",
    'name_action_epg_phase': 'define_epg_phases',
    }

#### Load data

In [5]:
project = expipe.get_project(p_dat['path_expipe'])

entities = list(project.entities)
actions = project.actions
actions_rec = {k: v for k, v in actions.items() if v.attributes['type'] == 'eeg recording'}

#### Create new action

In [6]:
action = project.require_action(p_dat['name_action_epg_phase'])
action.type = 'action_'+p_dat['name_action_epg_phase']

#### Define EPG phase separately for each animal


In [11]:
for idx in tqdm(entities[:1]):
    
    # get correspodning recording action
    action_i = [ac for k, ac in actions_rec.items() if idx in ac.entities]
    assert len(action_i) == 1
    action_i = action_i[0]
    
    # load annotation file
    fname_annot = action_i.data['df_data_annotation']
    path_annot = str(action_i.data_path()) + '/' + fname_annot
    annot_i = tools.load_dict(path_annot)


100% 1/1 [00:00<00:00,  1.00it/s]


In [15]:
annot_i

,Date,ID,group,Machine,Channel,Notes,EEG manually inspected,PPS Stimulation,Artifacts reported,Surgery,...,Seizure,Missing data,Conflict overlap recording,Device on,Device off,Spikes,Spindles,DBS location,DBS frequency,DBS stimulation
1755,2015-04-27,3212,4,yolo,11,"LC (EDF), OP",NaN,NaN,NaN,NaN,...,NaN,x,NaN,x,NaN,NaN,NaN,PP,130,NaN
1756,2015-04-28,3212,4,yolo,11,LC (EDF),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PP,130,NaN
1757,2015-04-29,3212,4,yolo,11,LC (EDF),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PP,130,NaN
1758,2015-04-30,3212,4,yolo,11,LC (EDF),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PP,130,NaN
1759,2015-05-01,3212,4,yolo,11,LC (EDF),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PP,130,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,2015-07-31,3212,4,yolo,11,NaN,NaN,NaN,NaN,NaN,...,NaN,x,NaN,NaN,NaN,NaN,NaN,PP,130,NaN
1851,2015-08-01,3212,4,yolo,11,NaN,NaN,NaN,NaN,NaN,...,NaN,x,NaN,NaN,NaN,NaN,NaN,PP,130,NaN
1852,2015-08-02,3212,4,yolo,11,NaN,NaN,NaN,NaN,NaN,...,NaN,x,NaN,NaN,NaN,NaN,NaN,PP,130,NaN
1853,2015-08-03,3212,4,yolo,11,NaN,NaN,NaN,NaN,NaN,...,NaN,x,NaN,NaN,NaN,NaN,NaN,PP,130,NaN


In [29]:
tools.define_epg_phases(
    annot_i,
    p_phase,
    p_columnnames)

AssertionError: 

In [ ]:
# group by id
gp_id = copy.deepcopy(df_recs).groupby('ID')

ls_res = []
for idx, gp_i in gp_id:
    try:
        res = tools.define_epg_phases(gp_i, p_phase)
        res['ID'] = idx
        ls_res.append(res)
    except Exception as e: 
        print('Skip animal: ' + str(idx))
        print(e)

df_epg_phase = pd.DataFrame(ls_res)
df_epg_phase = df_epg_phase.set_index('ID')

### Test table EPG phases

In [ ]:
ids = df_recs['ID'].unique()

dict_phase = copy.deepcopy(p_phase)
del dict_phase['Criteria_LatentPhase']
ls_keys = list(dict_phase.keys())

for i, row_i in df_epg_phase.iterrows():
    
    for key, val in dict_phase.items():
        if not pd.isnull(row_i[key+'_stop']):
            delta_t = row_i[key+'_stop'] - row_i[key+'_start']
            
            assert delta_t - p_phase[key]['len'] <= dt.timedelta(minutes=1)
        
    # make sure phases do not overlap    
    for j in range(len(ls_keys)-1):
        phase_j = ls_keys[j]
        phase_j_n = ls_keys[j+1]
        
        if (not pd.isnull(row_i[phase_j + '_stop']) and
            not pd.isnull(row_i[phase_j_n + '_start'])):  
            assert row_i[phase_j + '_stop'] < row_i[phase_j_n + '_start']

#### Add type of initial stimulation

In [ ]:
gp_id = df_recs.groupby('ID')

df_epg_phase['stimulation'] = np.nan

for idx, gp_i in gp_id:
    gp_i_sorted = gp_i.sort_values('datetime')
    stim = '_'.join(gp_i_sorted['Stimulation'].dropna().to_list())
    df_epg_phase.loc[idx, 'stimulation'] = stim

#### Define whether animals developed epilepsy

In [ ]:
df_epg_phase['epg'] = np.nan


# mark animals with spontaneous seizures
EPG_true = ~df_epg_phase['latent_stop'].isnull()
df_epg_phase.loc[EPG_true, 'epg'] = True

# mark those with spontaneous seizures

# add time between end of stimulation and end of latent phase
df_epg_phase['max_time_since_stimstart'] = (
    df_epg_phase['recording_last'] - df_epg_phase['stimulation_start'])

EPG_false = (
    (df_epg_phase['latent_stop'].isnull()) &
    (df_epg_phase['max_time_since_stimstart'] > p_EPG['max_time_latent']))
df_epg_phase.loc[EPG_false, 'epg'] = False

### Assign estimated phases for control animals
To allow comparison between animals which develop epilepsy and those who do not, we assign phases latent1->chronic2 for control animals based on average times in epg animals

In [ ]:
phases_to_assign =  ['Latent_1', 'Latent_2', 'Chronic_0', 'Chronic_1', 'Chronic_2']

In [ ]:
stim_stop = df_epg_phase['stimulation_stop']

dict_phase_start = {}
for phase_i in phases_to_assign:
    # get distances of phase start from animals which developed epilepsy
    d_start = df_epg_phase[df_epg_phase['epg']==True][phase_i + '_start']-stim_stop
    
    # get average phase start
    d_start_mean = d_start.mean()
    dict_phase_start[phase_i] = d_start_mean

In [ ]:
# attribute predefined phases to animals without epg
ids = df_epg_phase[df_epg_phase['epg']!=True].index

for idx in ids:
    stim_stop = df_epg_phase.loc[idx]['stimulation_stop']
    for key_i, val_i in dict_phase_start.items():
        len_phase_i = p_phase[key_i]['len']
        
        phase_i_start = stim_stop + val_i
        phase_i_stop = phase_i_start + len_phase_i
        
        # if any recording exists before terminating day, assign phase
        # phases can maximally last until day before perfusion
        rec_last_valid = df_epg_phase.loc[idx]['recording_last']-dt.timedelta(days=1)
        
        if phase_i_start <= rec_last_valid:
            df_epg_phase.loc[idx, key_i+'_start'] = phase_i_start
            phase_i_stop = np.minimum(phase_i_stop, rec_last_valid)
            df_epg_phase.loc[idx, key_i+'_stop'] = phase_i_stop         

#### Define treatment and control groups

In [ ]:
groups = {
    'PPS_EPG':{'epg': True, 'stimulation': '30min_30min_8h'},
    'PPS_noEPG':{'epg': False, 'stimulation': '30min_30min_8h'},
    'noPPS_noEPG':{'epg': False, 'stimulation': 'None_None_None'},
}
df_epg_phase['group̈́'] = None

for key_i, val_i in groups.items():
    bool_group_i = np.logical_and.reduce([df_epg_phase[key_j] == val_j for key_j, val_j in val_i.items()])
    df_epg_phase.loc[bool_group_i, 'group'] = key_i

In [ ]:
df_epg_phase['group']

#### Store features

In [ ]:
action_path = str(action.path)
fname = 'df_epg_phase.pckl'
path_fname_data = action_path + '/' + fname 
dect.save_dict(df_epg_phase, path_fname_data)
action.data[fname] = path_fname_data

### Visualize EPG phases

In [ ]:
ls_phase = [
    'BL_0',
    'BL_1',
    'Latent_0',
    'Latent_1',
    'Latent_2',
    'Chronic_0',
    'Chronic_1',
    'Chronic_2']

In [ ]:
df_sel = df_epg_phase[df_epg_phase['group'] == 'PPS_EPG']

In [ ]:
dt_surgery = df_sel['surgery']
dt_last = df_sel['recording_last']

#n_days = (dt_last - dt_surgery).max().days
n_days = 80
n_animals = len(df_sel)
ar_phase = np.zeros((n_animals, n_days))
ar_phase[:, :] = np.nan

for i, phase_i in enumerate(ls_phase):
    start_i = (df_sel[phase_i + '_start'] - dt_surgery).dt.days
    stop_i = (df_sel[phase_i + '_stop'] - dt_surgery).dt.days
    for j, idx_j in enumerate(df_sel.index.unique()):
        if ~np.isnan(start_i[idx_j]) and ~np.isnan(stop_i[idx_j]):
            start_ij = int(start_i[idx_j])
            stop_ij = int(stop_i[idx_j])
        ar_phase[j, start_ij:stop_ij+1] = i
    



In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 10])

n_phase = len(p_phase)
cmap = plt.get_cmap('viridis', n_phase)

img = ax.imshow(
    ar_phase,
    cmap=cmap,
    interpolation=None,
    vmax=n_phase-1)
ax.set_aspect('auto')

#bounds=np.array([0, 0.001, 0.01, 0.05, 1])
#norm = colors.BoundaryNorm(bounds, cmap.N)
cbar = plt.colorbar(
    img,
#    cmap=cmap,
    #norm=norm,
    #boundaries=bounds,
    #ticks=bounds[1:]/2.
)
cbar.ax.set_yticks(np.arange(0.5, n_phase-1, 1))
#cbar.ax.set_yticklabels(ls_phase)
#ax.set_title('p-value - Mann-Whitney-U')
#ax.set_xticks(np.arange(len(ls_phase)))
#ax.set_xticklabels(ls_phase)
#ax.tick_params(axis='x', rotation=45)
#ax.set_yticks(np.arange(len(ls_phase)))
#ax.set_yticklabels(ls_phase)

In [ ]:
ar_phase[3, :]

In [ ]:
n_days.days

In [ ]:
dt_surgery = df_epg_phase['surgery']
dt_last = df_epg_phase['recording_last']


In [ ]:
gp = df_epg_phase.groupby('id')
for i, (idx, gp_i) in gp:
    

In [ ]:
idx = 1218

In [ ]:
gp = df_epg_phase.groupby('id')

In [ ]:
gp_i = gp.get_group(idx)

In [ ]:
p_phase.keys()

In [ ]:
gp_i